# Agenda

- Rest of decorators
- Concurrency
    - threading
    - multiprocessing
- Final test    

In [2]:
import time
import random

class CalledTooSoonError(Exception):
    pass

def once_per_minute(func):         # decorator function gets one arg, the decorated function -- run once per decoration
    last_ran_at = 0
    def wrapper(*args):            # inner function ("wrapper") gets *args, is run once per call to the function
        nonlocal last_ran_at       # now, when we assign to last_ran_at, we change the enclosing var last_ran_at
        current_time = time.time()
        
        if current_time - last_ran_at < 60:
            raise CalledTooSoonError('Too soon!')
        
        result = func(*args)       # call the original function, with the args we got for it
        last_ran_at = current_time
        return result
    return wrapper


@once_per_minute
def slow_add(first, second):
    time.sleep(random.randint(0, 3))
    return first + second

@once_per_minute
def slow_mul(first, second):
    time.sleep(random.randint(0, 3))
    return first * second

print(slow_add(2, 3))
print(slow_mul(2, 3))
print(slow_add(4, 5))
print(slow_mul(6, 7))             

5
6


CalledTooSoonError: Too soon!

# Exercise: Memoization

Memoization means: When we call a function, we check the arguments:
    - If we have seen the arguments before, return the value from the previous run with those args.
    - If we *not* seen the arguments before, then call the function, and store the combination of args + result
    
Define a decorator, `memoize`, which implements memoization

In [4]:
def memoize(func):
    cache = {}
    def wrapper(*args):
        if args not in cache:
            cache[args] = func(*args)   # --> cache.__setitem__(args, func(*args))
            
        return cache[args]
    return wrapper


@memoize
def slow_add(first, second):
    time.sleep(random.randint(0, 3))
    print('Running slow_add')
    return first + second

@memoize
def slow_mul(first, second):
    time.sleep(random.randint(0, 3))
    print('Running slow_mul')
    return first * second

print(slow_add(2, 3))  # prints "running slow_add"
print(slow_mul(2, 3))  # prints "running slow_mul"
print(slow_add(2, 3))  # no printout (and faster result)
print(slow_mul(2, 3))  # no printout (and faster result)

Running slow_add
5
Running slow_mul
6
5
6


In [5]:
import time
import random

class CalledTooSoonError(Exception):
    pass

def once_per_n(n):                    # decorator, taking an argument
    def middle(func):                 # middle now takes the decorated function
        last_ran_at = 0
        def wrapper(*args):           # wrapper remains as it was before
            nonlocal last_ran_at 
            current_time = time.time()

            if current_time - last_ran_at < n:
                raise CalledTooSoonError(f'Too soon, wait another {n - (current_time - last_ran_at)}')

            result = func(*args)       # call the original function, with the args we got for it
            last_ran_at = current_time
            return result
        return wrapper
    return middle

@once_per_n(10)
def slow_add(first, second):
    time.sleep(random.randint(0, 3))
    return first + second

slow_add = once_per_n(10)(slow_add)

@once_per_n(5)
def slow_mul(first, second):
    time.sleep(random.randint(0, 3))
    return first * second

slow_mul = once_per_n(5)(slow_mul)

print(slow_add(2, 3))
print(slow_mul(2, 3))
print(slow_add(4, 5))
print(slow_mul(6, 7))             

5
6


CalledTooSoonError: Too soon, wait another 6.995551109313965

# Exercise: Only one type

1. Write a decorator, `only_ints`, that removes any arguments to the decorated function that are *not* integers.
2. The modify this decorator to be `only_of_type`, that takes an argument, a class.

In [10]:
def only_ints(func):
    def wrapper(*args):
        int_args = [one_arg
                   for one_arg in args
                   if isinstance(one_arg, int)]
        
        
        return func(*int_args)
    return wrapper

@only_ints
def mysum(*numbers):
    total = 0
    
    for one_number in numbers:
        total += one_number
        
    return total

mysum(10, 20, 'abcd', 30)   # 60

60

In [11]:
# we can rewrite it to be shorter!

def only_ints(func):
    def wrapper(*args):
        return func(*[one_arg
                       for one_arg in args
                       if isinstance(one_arg, int)])
    return wrapper

@only_ints
def mysum(*numbers):
    total = 0
    
    for one_number in numbers:
        total += one_number
        
    return total

mysum(10, 20, 'abcd', 30)   # 60

60

In [7]:
x = 10

if type(x) == int:
    print('Yes!')

Yes!


In [8]:
if isinstance(x, int):
    print('Yes!')

Yes!


In [21]:
def only_of_type(the_type):
    def middle(func):
        def wrapper(*args):
            return func(*[one_arg
                           for one_arg in args
                           if isinstance(one_arg, the_type)])
        return wrapper
    return middle


@only_of_type(int)
def mysum(*numbers):   # numbers is a tuple, containing all arguments
    print(f'{numbers=}')
    total = 0
    
    for one_number in numbers:
        total += one_number
    return total

# mysum = only_of_type(int)(mysum)


mysum(10, 20, 'abcd', 30)   # 60

numbers=(10, 20, 30)


60

In [ ]:
def shortest_word(*words):
    return min(words, key=len)  # sorts, then returns the first element -- like sorted, it takes a len

In [13]:
def add(first, second):
    return first + second

t = (10, 20)

add(t)

TypeError: add() missing 1 required positional argument: 'second'

In [14]:
add(*t)

30

In [17]:
mylist = [10, 20, 30]

'*'.join(str(one_item)
        for one_item in mylist)

'10*20*30'

In [23]:
def only_of_type(the_type):
    def middle(func):
        def wrapper(*args):
            return func(*[one_arg
                           for one_arg in args
                           if isinstance(one_arg, the_type)])
        return wrapper
    return middle


@only_of_type(int)
def mysum(*numbers):   # numbers is a tuple, containing all arguments
    """Sum numbers!
    
    Expects: Positional arguments, all numbers
    Modifies: Nothing
    Returns: Sum of numbers"""
    print(f'{numbers=}')
    total = 0
    
    for one_number in numbers:
        total += one_number
    return total

# mysum = only_of_type(int)(mysum)


mysum(10, 20, 'abcd', 30)   # 60

numbers=(10, 20, 30)


60

In [24]:
help(mysum)

Help on function wrapper in module __main__:

wrapper(*args)



In [25]:
# get the (a) function signature and (b) docstring back

from functools import wraps

def only_of_type(the_type):
    def middle(func):

        @wraps(func)   # assign to wrapper the signature + docstring from func
        def wrapper(*args):
            return func(*[one_arg
                           for one_arg in args
                           if isinstance(one_arg, the_type)])
        return wrapper
    return middle


@only_of_type(int)
def mysum(*numbers):   # numbers is a tuple, containing all arguments
    """Sum numbers!
    
    Expects: Positional arguments, all numbers
    Modifies: Nothing
    Returns: Sum of numbers"""
    print(f'{numbers=}')
    total = 0
    
    for one_number in numbers:
        total += one_number
    return total

# mysum = only_of_type(int)(mysum)


mysum(10, 20, 'abcd', 30)   # 60

numbers=(10, 20, 30)


60

In [26]:
help(mysum)

Help on function mysum in module __main__:

mysum(*numbers)
    Sum numbers!
    
    Expects: Positional arguments, all numbers
    Modifies: Nothing
    Returns: Sum of numbers



# Concurrency in Python (threads and processes)

In [27]:
# GIL -- global interpreter lock

In [28]:
import sys


In [29]:
sys.getswitchinterval()

0.005

# Exercise: File lengths

1. Write a function, `file_length`, that takes a filename (string) as an input, and returns the length of the file.  Iterate over the file, one line at a time, to get its length.
2. Use `concurrent.futures` and `ThreadPoolExecutor` to run this function on each text file in a directory.
3. How many bytes, total, are there in those files?

In [31]:
list(map(len, 'this is a test'.split()))

[4, 2, 1, 4]

In [32]:
sys.getswitchinterval()

0.005

In [33]:
help(sys.setswitchinterval)

Help on built-in function setswitchinterval in module sys:

setswitchinterval(interval, /)
    Set the ideal thread switching delay inside the Python interpreter.
    
    The actual frequency of switching threads can be lower if the
    interpreter executes long sequences of uninterruptible code
    (this is implementation-specific and workload-dependent).
    
    The parameter must represent the desired switching delay in seconds
    A typical value is 0.005 (5 milliseconds).

